# Simulation #1
This workbook is a first attempt at constructing a simulation of a stochastic dynamical system, with an accompanying estimation of the Perron-Frobenius operator.

In [1]:
using LinearAlgebra, Plots, Distributions

In [2]:
plotly();

┌ Info: For saving to png with the Plotly backend PlotlyBase has to be installed.
└ @ Plots /Users/ryancushen/.julia/packages/Plots/iYDwd/src/backends.jl:372


## Setting up the dynamics
The state space $X$ of our dynamical system will be the flat torus $\mathbb{T}^2$, represented by the unit square.

In [3]:
grid_size = 100;
n_gridpoints = grid_size ^ 2;

In [4]:
xs = LinRange(0, 1, grid_size);
ys = LinRange(0, 1, grid_size);
grid = [[x, y] for x in xs for y in ys];
grid = transpose(hcat(grid...));

We will define an initial density $f_0 (x, y)$ on this state space, which will be the sum of four independent normal distributions spaced evenly in the unit square.

In [5]:
function f0(x)
    M = [0.25 0.25; 0.75 0.25; 0.25 0.75; 0.75 0.75]
    σ = 1/50
    Σ = σ .* [1 0; 0 1]
    weight = 1 / 4
    value = 0
    for i in 1:4
        d = MvNormal(M[i,:], Σ)
        eval = pdf(d, x)
        value += weight * eval
    end
    return value
end;

In [6]:
f0s = [f0(grid[n,:]) for n in 1:n_gridpoints]
scatter(grid[:,1], grid[:,2], f0s, markersize=1)

<!DOCTYPE html>
 
 
 Plots.jl

To obtain a sample from this initial density $f_0$, we can use rejection sampling.

In [7]:
n_samples = 10000
candidate_points = rand(n_samples,2)
f0s = [f0(candidate_points[n,:]) for n in 1:n_samples]
thresholds = rand(n_samples)
sample = candidate_points[f0s .> thresholds, :]
histogram2d(sample[:,1], sample[:,2], bins=30)

<!DOCTYPE html>
 
 
 Plots.jl

In [8]:
sample_size = size(sample, 1)

7679

The particular map $S: \mathbb{T}^2 \to \mathbb{T}^2$ we choose to introduce the dynamics is given by
$$
S ([x , y]) := \begin{bmatrix} x + y \\ y + a \sin ( x + y) \end{bmatrix} \mod 1
$$
where $a$ is a parameter which for the moment we take to be $a=4$. This map is easily wrapped in a function which can then be iterated forward across timesteps.

In [9]:
function S(x)
    a = 4;
    result = [x[:,1]+x[:,2] x[:,2] + a*sin.(x[:,1]+x[:,2])];
    result = mod.(result,1)
    return result
end;

function S_forward(initial_points, steps)
    x = initial_points
    for t in 1:steps
        x = S(x)
    end
    return x
end;

We can then easily compare the initial density $f_0$ with a naive estimate of a limiting density $f_*$ by computing the latter with a big value for ```S_forward```.

In [10]:
fstar = S_forward(sample, 100);

In [11]:
histogram2d(fstar[:,1], fstar[:,2], bins=50)

<!DOCTYPE html>
 
 
 Plots.jl

## Estimating the Perron-Frobenius operator
Having established a model for the dynamics, next we want to estimate the Perron-Frobenius operator. 
$$
\mathcal{P} : L^1(X) \to L^1(X)
$$
Actually we will be estimating a stochastically perturbed version of $\mathcal{P}$, described by
$$
 \mathcal{L} f(y) = \int_X k(Sx, y ) f(x) dx
$$
where $k(x, y) = \phi(x-y)$.

To estimate $\mathcal{L}$, we first must define a finite basis for $L^1(X)$. For the moment we will take this basis to be a uniform grid of radial basis functions (RBFs) $\varphi_i$. These will be of the form
$$
\varphi_i (x) = \phi ( x - z_i ) := \exp \left( - \frac{\| x - z_i \|^2}{\epsilon^2} \right)
$$
where the $z_i$ denote the centres of each RBF and $\epsilon$ is some bandwidth parameter.

In [12]:
φ(x, z, ϵ) =  exp( -1 * ( norm(x - z) / ϵ ) ^ 2 );

In [13]:
ϵ = 0.2;

In [14]:
basis_grid_size = 25;
n_bases = basis_grid_size ^ 2;

In [15]:
bxs = LinRange(0, 1, basis_grid_size);
bys = LinRange(0, 1, basis_grid_size);
basis_locs = [[x, y] for x in bxs for y in bys];
basis_locs = transpose(hcat(basis_locs...));

These basis functions can be easily visualised by evaluating all grid points against each.

In [16]:
evaluation_matrix = zeros(n_gridpoints, n_bases)
for b in 1:n_bases
    for i in 1:n_gridpoints
        evaluation_matrix[i, b] = φ(grid[i, :], basis_locs[b, :], ϵ)
    end
end

In [17]:
scatter(grid[:,1], grid[:,2], evaluation_matrix[:,270], markersize=1)
scatter!(grid[:,1], grid[:,2], evaluation_matrix[:,1106], markersize=1)

LoadError: BoundsError: attempt to access 10000×625 Matrix{Float64} at index [1:10000, 1106]

In order to estimate the integral
$$
    \mathcal{L} f(y) = \int_X k (Sx, y) f(x) dx
$$
we will also need an approximate Lebesgue measure for the integral wrt $dx$. This will be some weighted combination of all the datapoints, subject to the constraint that obviously each basis function integrates to a constant. This requires computing an evaluation matrix $\Phi$ of all points against all basis functions.

In [18]:
Φ = zeros(n_bases, sample_size)
for j in 1:n_bases
    for i in 1:sample_size
        evaluation = φ(sample[i, :], basis_locs[j, :], ϵ)
        Φ[j, i] = evaluation
    end
end

Having done so, we can then estimate the weights $w$ using nonnegative least squares.

In [19]:
c = π * ϵ^2;
C = c * ones(n_bases);

In [20]:
include("nnlsq.jl");

In [21]:
w, residual, objvalue = nnlsq(Φ, C, 0);

Academic license - for non-commercial use only - expires 2021-08-05
Warning for adding constraints: zero or small (< 1e-13) coefficients, ignored
Gurobi Optimizer version 9.1.2 build v9.1.2rc0 (mac64)
Thread count: 2 physical cores, 4 logical processors, using up to 4 threads
Optimize a model with 625 rows, 8304 columns and 4766945 nonzeros
Model fingerprint: 0x7d4b4d3f
Model has 625 quadratic objective terms
Coefficient statistics:
  Matrix range     [1e-13, 1e+00]
  Objective range  [0e+00, 0e+00]
  QObjective range [2e+00, 2e+00]
  Bounds range     [0e+00, 0e+00]
  RHS range        [1e-01, 1e-01]
Presolve time: 1.27s
Presolved: 625 rows, 8304 columns, 4766945 nonzeros
Presolved model has 625 quadratic objective terms
Ordering time: 0.00s

Barrier statistics:
 AA' NZ     : 1.950e+05
 Factor NZ  : 1.956e+05 (roughly 5 MBytes of memory)
 Factor Ops : 8.158e+07 (less than 1 second per iteration)
 Threads    : 2

                  Objective                Residual
Iter       Primal      

With these weights, we can now write the image of any basis function $\varphi_i$ under $\mathcal{L}$ as a linear combination of kernels centered at the image points, denoted $\varphi_{y_j}$. But we obviously want to be able to write $\mathcal \varphi_i$ as just a linear combination of the $\varphi_i$. To do this, we will need to write $\varphi_{y_j}$ as a linear combination of the $\varphi_i$.
$$
 \varphi_{y_j} (y) = \sum_{i=1}^m \gamma_i \varphi_i (y)
$$
The weights $\gamma_i$ will be calculated as the integrals of $\varphi_{y_j}$ over a Voronoi tesselation of the basis functions $\varphi_i$, weighted according to $c_i$.

In [22]:
X = sample;
Y = S(X);

In particular, we will need to integrate $\varphi_{y_j}$ over a Voronoi tesselation of $X$.

In [80]:
#function integrate_phiyj(y_j, zs)
    # integrates φ at y_j across a Voronoi tesselation over m basis functions with centres zs
    # returns the m integral values

y_j = [0.2 0.4]
zs = basis_locs

fine_grid_size = 100;
n_fine_gridpoints = fine_grid_size ^ 2;

fine_xs = LinRange(0, 1, fine_grid_size);
fine_ys = LinRange(0, 1, fine_grid_size);
fine_grid = [[x, y] for x in fine_xs for y in fine_ys];
fine_grid = transpose(hcat(fine_grid...));

distance_matrix = zeros(n_fine_gridpoints, n_bases)

for g in 1:n_fine_gridpoints
    for b in 1:n_bases
       dist = norm(fine_grid[g,:] - zs[b,:])
       distance_matrix[g,b] = dist
    end
end

inds = argmin(distance_matrix, dims=2);
vs = [ind[2] for ind in inds]

#end;

#integrate_phiyj([0.2 0.4], basis_locs)

In [108]:
integral_values = zeros(n_bases)
for b in 1:n_bases
    # get all the fine_grid points which have vs = b; this forms V_k
    V_k = these points
    evaluation = 0
    for i in these points
        evaluations += φ(V_k[i], y_j, ϵ)
    end
    result = 1/n * evaluations
    integral_values[b] = result
end

LoadError: syntax: "for" at In[108]:2 expected "end", got "points"